In [3]:
!pip install langchain
!pip install huggingface_hub
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
# !pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cac

In [4]:
from langchain import PromptTemplate,HuggingFaceHub,LLMChain
import os

In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""

# Seq2Seq Model

In [6]:
prompt=PromptTemplate(input_variables=["product"],
                      template="what is a good name of a company that makes {product}")
chain=LLMChain(llm=HuggingFaceHub(repo_id="google/flan-t5-base",model_kwargs={'temperature':0.9,"max_length":40}),prompt=prompt)
chain.run("Ice Cream")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'swiss'

In [15]:
prompt=PromptTemplate(input_variables=["name"],
                      template="Describe me about famous personality {name} in essay")
chain=LLMChain(llm=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={'temperature':0.1}),prompt=prompt)
chain.run(name="Messi")

'he is a soccer player'

In [17]:
prompt=PromptTemplate(input_variables=["name"],
                      template="Tell me about famous footballer {name}")
chain=LLMChain(llm=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={'temperature':0.1}),prompt=prompt)
chain.run(name="Messi")

'Messi is a footballer who plays for FC Barcelona.'

# Text Generation Model

In [22]:
prompt=PromptTemplate(input_variables=["name"],
                      template="Tell me about famous footballer {name}")
chain=LLMChain(llm=HuggingFaceHub(repo_id="tiiuae/falcon-7B",model_kwargs={'temperature':0.1,"max_length":50}),prompt=prompt)
print(chain.run(name="Messi"))

Tell me about famous footballer Messi
Answers:
Lionel Messi is a footballer who plays for the Argentine national team. He is a forward and is considered to be one of the best players in the world. He plays for the Spanish club FC Barcelona.
He is 5'7" tall and weighs 150 pounds. He was born in Rosario, Argentina on June 24, 1987.
He is the son of Jorge Messi, a former footballer who played for the Argentinian club New


# TO download Model locally and Run

In [30]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


In [31]:
model_id = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')
pipelines = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=128)
local_llm = HuggingFacePipeline(pipeline=pipelines)
prompt = PromptTemplate(input_variables=["product"],
    template="What is a good name for a company that makes {product}")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [35]:
chain = LLMChain(llm=local_llm,prompt = prompt)
chain.run("Socks")


'sock factory'